In [ ]:
import pandas as pd, numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler , OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from lazypredict.Supervised import LazyRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import RandomizedSearchCV, KFold

In [18]:
df = pd.read_csv('winequality-red.csv')

In [19]:
X = df.drop(columns = "quality")
y = df['quality']

In [20]:
X_train , X_test , y_train , y_test = train_test_split(
    X , y , random_state = 42 , test_size = 0.2 , stratify = y
)

In [21]:
reg = LazyRegressor(verbose=0,ignore_warnings=False, custom_metric=None )


In [22]:
models,predictions = reg.fit(X_train, X_test, y_train, y_test)

  0%|          | 0/42 [00:00<?, ?it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000172 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1023
[LightGBM] [Info] Number of data points in the train set: 1279, number of used features: 11
[LightGBM] [Info] Start training from score 5.637217


In [23]:
models, predictions

(                               Adjusted R-Squared  R-Squared  RMSE  Time Taken
 Model                                                                         
 ExtraTreesRegressor                          0.51       0.53  0.55        0.11
 HistGradientBoostingRegressor                0.48       0.50  0.57        0.37
 RandomForestRegressor                        0.48       0.50  0.57        0.29
 LGBMRegressor                                0.47       0.49  0.57        0.33
 BaggingRegressor                             0.41       0.43  0.61        0.03
 XGBRegressor                                 0.41       0.43  0.61        0.20
 GradientBoostingRegressor                    0.40       0.42  0.61        0.13
 SVR                                          0.39       0.41  0.62        0.03
 NuSVR                                        0.38       0.41  0.62        0.03
 AdaBoostRegressor                            0.38       0.40  0.62        0.07
 HuberRegressor                         

In [24]:
model = ExtraTreesRegressor(random_state=42, n_jobs=-1)

param_dist = {
    "n_estimators": [300, 600, 1000],
    "max_depth": [None, 10, 20, 40],
    "min_samples_leaf": [1, 2, 5, 10],
    "min_samples_split": [2, 5, 10],
    "max_features": ["sqrt", 0.5, 0.8, 1.0],
}

cv = KFold(n_splits=5, shuffle=True, random_state=42)

search = RandomizedSearchCV(
    model,
    param_distributions=param_dist,
    n_iter=30,
    scoring="neg_root_mean_squared_error",
    cv=cv,
    random_state=42,
    n_jobs=-1,
    verbose=1
)

search.fit(X_train, y_train)

best = search.best_estimator_
pred = best.predict(X_test)

print("Best params:", search.best_params_)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


/Users/elihuseyn/anaconda3/envs/pervi/lib/python3.11/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/Users/elihuseyn/anaconda3/envs/pervi/lib/python3.11/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/Users/elihuseyn/anaconda3/envs/pervi/lib/python3.11/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain

Best params: {'n_estimators': 1000, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 0.5, 'max_depth': 40}


In [25]:
y_pred = best.predict(X_test)

compare = pd.DataFrame({
    "y_true": y_test.values if hasattr(y_test, "values") else y_test,
    "y_pred": y_pred,
})

compare["error"] = compare["y_true"] - compare["y_pred"]          
compare["abs_error"] = compare["error"].abs()
compare["pct_error_%"] = 100 * compare["error"] / (compare["y_true"].replace(0, np.nan))

compare_sorted = compare.sort_values("abs_error", ascending=False)

display(compare_sorted.head(100))   

,y_true,y_pred,error,abs_error,pct_error_%
144,3,5.27,-2.27,2.27,-75.73
186,8,6.11,1.89,1.89,23.64
221,3,4.87,-1.87,1.87,-62.23
293,4,5.68,-1.68,1.68,-41.95
49,7,5.37,1.63,1.63,23.33
...,...,...,...,...,...
87,6,5.46,0.54,0.54,9.08
41,5,5.54,-0.54,0.54,-10.88
89,6,5.46,0.54,0.54,8.97
256,5,5.54,-0.54,0.54,-10.74


In [26]:
print("R2:", r2_score(compare["y_true"], compare["y_pred"]))
print("MAE:", compare["abs_error"].mean())
print("Median AE:", compare["abs_error"].median())
print("Max AE:", compare["abs_error"].max())

R2: 0.5362442617591865
MAE: 0.368134375
Median AE: 0.23150000000000004
Max AE: 2.2720000000000002
